# Estrutura de Dados da SuperStore

## Descrição do problema

Super Store, líder no setor varejista, enfrenta o desafio de gerenciar grandes volumes de dados dispersos. Para melhorar a tomada de decisões, propõe-se a implementação de um sólido sistema ETL com tabelas de fatos e dimensões. Este projeto busca criar um sistema abrangente que extraia, transforme e carregue dados de forma eficiente, proporcionando uma estrutura hierárquica para facilitar a análise. O objetivo não é apenas otimizar o armazenamento, mas também potencializar a capacidade da Super Store para identificar padrões e oportunidades de mercado, adaptando-se ágilmente a mudanças na demanda do consumidor.

```
[CSV]
    ↓
   Python (jupyter notebook)
    ↳ Limpeza conceitual (E - Extract)
    ↳ Padronização (T - Transform)
        ↓
 [DataFrame limpo]
        ↓
   BigQuery (L - Load)
    ↳ Criação de tabelas fato e dimensão
```

### Descrição das variáveis

| Variável | Descrição |
| -------- | --------- |
| category | Representa as categorias de produtos vendidos no hipermercado. |
| city | Representa a cidade onde o pedido foi feito. |
| country | Representa o país onde o hipermercado está localizado. |
| customer_id | Representa um identificador único para cada cliente. |
| customer_name | Representa o nome do cliente que fez o pedido. |
| discount | Representa o desconto aplicado no pedido. |
| market | Representa o mercado ou região onde o hipermercado atua. |
| unknown | Uma coluna desconhecida ou não especificada. |
| order_date | Representa a data em que o pedido foi feito. |
| order_id | Um identificador único para cada pedido. |
| order_priority | Representa o nível de prioridade do pedido. |
| product_id | Representa um identificador exclusivo para cada produto. |
| product_name | Representa o nome do produto. |
| profit | Representa o lucro gerado pelo pedido. |
| quantity | Representa a quantidade de produtos encomendados. |
| region | Representa a região onde o pedido foi feito. |
| row_id | Representa um identificador exclusivo para cada linha do conjunto de dados. |
| sales | Representa o valor total de vendas do produto no pedido. |
| segment | Representa o segmento do cliente (por exemplo, consumidores, empresas ou escritórios domésticos). |
| ship_date | Representa a data em que o pedido foi enviado. |
| ship_mode | Representa o modo de envio usado para o pedido. |
| shipping_cost | Representa o custo de envio do pedido. |
| state | Representa o estado ou região do país. |
| sub_category | Representa a subcategoria de produtos dentro da categoria principal. |
| year | Representa o ano em que o pedido foi feito. |
| market2 | Outra coluna relacionada a informações de mercado. |
| weeknum | Representa o número da semana em que o pedido foi feito. |

## 1. Imports e configurações

In [34]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from google.cloud import bigquery
from pandas_gbq import to_gbq
import re

In [35]:
df = pd.read_csv('../data/superstore.csv')

## 2. Desenvolvimento

### a. Nulos e duplicatas

In [36]:
null_values = df.isnull().sum().reset_index()
null_values.columns = ['Variável', 'Qtd_Nulos']
null_values

,Variável,Qtd_Nulos
0,category,0
1,city,0
2,country,0
3,customer_ID,0
4,customer_name,0
5,discount,0
6,market,0
7,unknown,0
8,order_date,0
9,order_id,0


In [37]:
variable_report = null_values[null_values['Qtd_Nulos'] > 0]

In [38]:
duplicated_values = df[df.duplicated()]
print(f"Total de registros duplicados: {len(duplicated_values)}")

Total de registros duplicados: 0


In [39]:
df['row_id'].duplicated().sum()

np.int64(0)

### b. Variáveis categóricas

In [40]:
df.columns = df.columns.str.lower()

In [41]:
cat_cols = df.select_dtypes(include='object').columns

for col in cat_cols:
    df[col] = df[col].str.strip().str.lower()

In [42]:
for col in cat_cols:
    unique_categories = df[col].unique()
    print(f"\n{col} == {len(unique_categories)}")
    if len(unique_categories) < 20:
        print(unique_categories)


category == 3
['office supplies' 'technology' 'furniture']

city == 3636

country == 147

customer_id == 4873

customer_name == 795

market == 7
['us' 'eu' 'latam' 'africa' 'apac' 'emea' 'canada']

order_date == 1430

order_id == 25035

order_priority == 4
['high' 'medium' 'critical' 'low']

product_id == 10292

product_name == 3788

region == 13
['west' 'east' 'south' 'central' 'africa' 'central asia' 'north asia'
 'caribbean' 'north' 'emea' 'oceania' 'southeast asia' 'canada']

segment == 3
['consumer' 'home office' 'corporate']

ship_date == 1464

ship_mode == 4
['second class' 'standard class' 'same day' 'first class']

state == 1094

sub_category == 17
['paper' 'art' 'storage' 'appliances' 'supplies' 'envelopes' 'fasteners'
 'labels' 'binders' 'accessories' 'phones' 'copiers' 'machines' 'tables'
 'bookcases' 'chairs' 'furnishings']

market2 == 6
['north america' 'eu' 'latam' 'africa' 'apac' 'emea']


### c. Variáveis numéricas

In [43]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

for col in num_cols:
    invalid_values = df[~df[col].apply(lambda x: np.isreal(x)) & df[col].notnull()]
    if not invalid_values.empty:
        print(f"Inconsistências encontradas em {col}:")
        display(invalid_values)

In [44]:
outliers_dict = {}

for col in num_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    limits = (q1 - 1.5*iqr, q3 + 1.5*iqr)
    outliers = df[(df[col] < limits[0]) | (df[col] > limits[1])]
    print(f"{col}: {len(outliers)} outliers identificados.")
    outliers_dict[col] = len(outliers)


discount: 4172 outliers identificados.
unknown: 0 outliers identificados.
profit: 9755 outliers identificados.
quantity: 877 outliers identificados.
row_id: 0 outliers identificados.
sales: 5655 outliers identificados.
shipping_cost: 5909 outliers identificados.
year: 0 outliers identificados.
weeknum: 0 outliers identificados.


In [45]:
outliers_df = pd.DataFrame(list(outliers_dict.items()), 
                           columns=['Variável', 'Qtd_Outliers'])
variable_report = null_values.merge(outliers_df, on='Variável', how='outer')
variable_report = variable_report[(variable_report['Qtd_Nulos'] > 0) | 
                                   (variable_report['Qtd_Outliers'] > 0)]
variable_report = variable_report.fillna(0)
variable_report['Qtd_Nulos'] = variable_report['Qtd_Nulos'].astype(int)
variable_report['Qtd_Outliers'] = variable_report['Qtd_Outliers'].astype(int)

### d. Garantir tipos de dados corretos

In [46]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['ship_date'] = pd.to_datetime(df['ship_date'], errors='coerce')
df['customer_id'] = df['customer_id'].astype(str)
df['order_id'] = df['order_id'].astype(str)

### e. Pesquisar dados de outras fontes

In [47]:
wiki = "https://en.wikipedia.org/wiki/List_of_supermarket_chains"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
req = Request(wiki, headers=headers)
wiki_page_multinacional = urlopen(req)

soup = BeautifulSoup(wiki_page_multinacional, 'html.parser')

wiki_page_multinacional.close()

In [48]:
print(soup.title.string)

List of supermarket chains - Wikipedia


In [49]:
tabela = soup.find('table', class_='wikitable')

In [50]:
dados = []
linhas = tabela.find_all('tr')

cabecalho = []
for th in linhas[0].find_all('th'):
    cabecalho.append(th.text.strip())

for linha in linhas[1:]:
    colunas = linha.find_all(['td', 'th'])
    if len(colunas) > 0:
        linha_dados = []
        for coluna in colunas:
            linha_dados.append(coluna.text.strip())
        dados.append(linha_dados)

df_multinacional = pd.DataFrame(dados, columns=cabecalho)
df_multinacional.columns = df_multinacional.columns.str.lower()
df_multinacional = df_multinacional.drop(columns=['map'])

In [51]:
multinacional_rename_columns = {
    "served countries (besides the headquarters)" : "countries",
    "number of locations": "locations",
    "number of employees": "employees"
}

df_multinacional = df_multinacional.rename(columns=multinacional_rename_columns)

In [52]:
def extract_number(value):
    if pd.isna(value) or value == '':
        return None
    numbers = re.sub(r'[^\d]', '', str(value))
    return int(numbers) if numbers else None

df_multinacional['locations'] = df_multinacional['locations'].apply(extract_number)
df_multinacional['employees'] = df_multinacional['employees'].apply(extract_number)

df_multinacional['countries_count'] = df_multinacional['countries'].apply(
    lambda x: len([country.strip() for country in str(x).split(',') if country.strip() and country.strip() != 'nan']) if pd.notna(x) and str(x).strip() != '' else 1
)

### f. Desenhar estrutura da base de dados

<img src="../results/tabelas_fato_dimensao.png" atl="estrutura_dos_dados" height="400">

### g. Criar estrutura da base de dados

In [53]:
dim_tempo = df[['order_date']].drop_duplicates().copy()
dim_tempo['date_id'] = dim_tempo['order_date'].dt.strftime('%Y%m%d').astype(int)
dim_tempo['year'] = dim_tempo['order_date'].dt.year
dim_tempo['weeknum'] = dim_tempo['order_date'].dt.isocalendar().week
dim_tempo = dim_tempo[['date_id', 'order_date', 'year', 'weeknum']]

In [54]:
dim_localizacao = df[['region', 'country', 'state', 'city', 'market', 'market2']].drop_duplicates().copy()
dim_localizacao['location_id'] = range(1, len(dim_localizacao) + 1)
dim_localizacao = dim_localizacao[['location_id', 'region', 'country', 'state', 'city', 'market', 'market2']]

In [55]:
dim_envio = df[['order_id', 'ship_date', 'ship_mode', 'shipping_cost']].drop_duplicates().copy()
dim_envio = dim_envio[['order_id', 'ship_date', 'ship_mode', 'shipping_cost']]

In [56]:
dim_cliente = df[['customer_id', 'customer_name', 'segment']].drop_duplicates()

In [57]:
dim_produto = df[['product_id', 'product_name', 'category', 'sub_category']].drop_duplicates()

In [58]:
fato_vendas = df[['row_id', 'order_id', 'customer_id', 'product_id', 
                   'order_date', 'region', 'country', 'state', 'city', 'market', 'market2',
                   'sales', 'profit', 'quantity', 'discount', 'order_priority']].copy()

fato_vendas['date_id'] = fato_vendas['order_date'].dt.strftime('%Y%m%d').astype(int)

fato_vendas = fato_vendas.merge(
    dim_localizacao[['location_id', 'region', 'country', 'state', 'city', 'market', 'market2']],
    on=['region', 'country', 'state', 'city', 'market', 'market2'],
    how='left'
)

fato_vendas = fato_vendas[['row_id', 'order_id', 'customer_id', 'product_id', 
                           'date_id', 'location_id', 'order_priority',
                           'sales', 'profit', 'quantity', 'discount']]

In [59]:
dim_company = df_multinacional.copy()
dim_company['company_id'] = range(1, len(dim_company) + 1)
dim_company = dim_company[['company_id', 'company', 'headquarters', 'countries', 'countries_count', 'locations', 'employees']]
dim_company

,company_id,company,headquarters,countries,countries_count,locations,employees
0,1,7-Eleven,JapanUnited States,"Australia, Canada, China, Cambodia, Denmark, H...",17,8.500000e+04,170000.0
1,2,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",15,2.000800e+04,560000.0
2,3,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",14,7.659000e+03,375000.0
3,4,Aldi,Germany (Süd and Nord),"Australia (Süd), Austria (Süd as Hofer), China...",18,1.283213e+09,274172.0
4,5,Edeka,Germany,Denmark,1,1.364600e+04,381000.0
...,...,...,...,...,...,...,...
366,367,T&T Supermarket,Canada,,1,3.300000e+01,NaN
367,368,Match,Belgium,"Luxembourg, France",2,2.170000e+02,NaN
368,369,C-market,Serbia,,1,NaN,NaN
369,370,Tegut,Germany,,1,2.750000e+02,7700.0


### g. Programar atualizações da tabela

#### Boas práticas e pontos-chave

- Usar tabelas de staging: carregar os novos registros em staging, validar, deduplicar, e somente depois fazer MERGE nas tabelas finais.
- Idempotência: esquemas de MERGE garantem que re-executar o pipeline não crie duplicatas — usar chaves naturais ou `row_id`/business key.
- SCD (tipo de dimensão): decidir SCD1 (overwrite) ou SCD2 (historical) para cada dimensão; por exemplo, `dim_cliente` pode ser SCD2 para rastrear mudanças de endereço/segmento.
- Chaves: definir chaves naturais (product_id, customer_id, order_id+product_id ou row_id) e chaves surrogate (integers) onde necessário.

#### Exemplo de MERGE (BigQuery) para dim_produto:

```sql
MERGE `project.dataset.dim_produto` T
USING `project.dataset.stg_produto` S
ON T.product_id = S.product_id
WHEN MATCHED AND (T.product_name != S.product_name OR T.category != S.category OR T.sub_category != S.sub_category) THEN
  UPDATE SET product_name = S.product_name, category = S.category, sub_category = S.sub_category
WHEN NOT MATCHED THEN
  INSERT (product_id, product_name, category, sub_category) VALUES (S.product_id, S.product_name, S.category, S.sub_category);
```

#### Exemplo de MERGE (BigQuery) para fato_vendas (upsert por `row_id`):

```sql
MERGE `project.dataset.fato_vendas` F
USING `project.dataset.stg_vendas` S
ON F.row_id = S.row_id
WHEN MATCHED THEN
  UPDATE SET sales = S.sales, profit = S.profit, quantity = S.quantity, discount = S.discount
WHEN NOT MATCHED THEN
  INSERT (row_id, order_id, customer_id, product_id, date_id, location_id, order_priority, sales, profit, quantity, discount)
  VALUES (S.row_id, S.order_id, S.customer_id, S.product_id, S.date_id, S.location_id, S.order_priority, S.sales, S.profit, S.quantity, S.discount);
```

#### Exemplo simples de upsert em pandas (staging -> target):

```python
# stg: novos registros, target: tabela já existente (dataframes)
key = 'row_id'
target = target.set_index(key)
stg = stg.set_index(key)
# atualizar/mesclar registros existentes e adicionar novos
target.update(stg)
new_rows = stg.index.difference(target.index)
if len(new_rows):
    target = pd.concat([target, stg.loc[new_rows]])
target = target.reset_index()
```

#### Checklist para marcar esse checkpoint como cumprido:

- [x] Sequência de atualização definida e documentada (esta célula).
- [ ] Implementar staging + MERGE/upsert (opcional neste projeto).
- [ ] Agendar/automação usando ferramenta escolhida (opcional neste checkpoint).

### h. Carga no BigQuery (Load)

In [60]:
load_dotenv()

credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if not credentials_path:
    raise ValueError("GOOGLE_APPLICATION_CREDENTIALS não está configurada no .env")

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

project_id = os.getenv("PROJECT_ID")
if not project_id:
    raise ValueError("PROJECT_ID não está configurado no .env")

dataset_name = "superstore"

def upload_to_bigquery(dataframe, table_name, project_id, if_exists='replace'):
    try:
        print(f"Iniciando upload para {table_name}...")
        dataframe.to_gbq(
            destination_table=table_name,
            project_id=project_id,
            if_exists=if_exists,
            progress_bar=False
        )
        print(f"✓ Upload concluído: {table_name} ({len(dataframe)} registros)")
    except Exception as e:
        print(f"✗ Erro ao fazer upload de {table_name}: {str(e)}")
        raise

upload_to_bigquery(
    fato_vendas, 
    f"{dataset_name}.fato_vendas", 
    project_id
)

dimensoes = {
    'dim_tempo': dim_tempo,
    'dim_localizacao': dim_localizacao,
    'dim_envio': dim_envio,
    'dim_cliente': dim_cliente,
    'dim_produto': dim_produto
}

for table_name, dataframe in dimensoes.items():
    upload_to_bigquery(
        dataframe,
        f"{dataset_name}.{table_name}",
        project_id
    )

upload_to_bigquery(
    dim_company,
    "dataset_wiki.dim_company",
    project_id
)

print("Todos os uploads foram concluídos com sucesso!")

Iniciando upload para superstore.fato_vendas...


C:\Users\Vitoria\AppData\Local\Temp\ipykernel_34108\2720867882.py:18: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  dataframe.to_gbq(


✓ Upload concluído: superstore.fato_vendas (51290 registros)
Iniciando upload para superstore.dim_tempo...


C:\Users\Vitoria\AppData\Local\Temp\ipykernel_34108\2720867882.py:18: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  dataframe.to_gbq(


✓ Upload concluído: superstore.dim_tempo (1430 registros)
Iniciando upload para superstore.dim_localizacao...


...

✓ Upload concluído: superstore.dim_cliente (4873 registros)
Iniciando upload para superstore.dim_produto...


C:\Users\Vitoria\AppData\Local\Temp\ipykernel_34108\2720867882.py:18: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  dataframe.to_gbq(


✓ Upload concluído: superstore.dim_produto (10768 registros)
Iniciando upload para dataset_wiki.tb_multinacional...


C:\Users\Vitoria\AppData\Local\Temp\ipykernel_34108\2720867882.py:18: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  dataframe.to_gbq(


✓ Upload concluído: dataset_wiki.tb_multinacional (371 registros)
Todos os uploads foram concluídos com sucesso!
